In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from itertools import product

In [2]:
# 1. Load data
events = pd.read_csv('data/events.csv')
games_info = pd.read_csv('data/ginf.csv')

In [3]:
events

,id_odsp,id_event,sort_order,time,text,event_type,event_type2,side,event_team,opponent,...,player_in,player_out,shot_place,shot_outcome,is_goal,location,bodypart,assist_method,situation,fast_break
0,UFot0hit/,UFot0hit1,1,2,Attempt missed. Mladen Petric (Hamburg) left f...,1,12.0,2,Hamburg SV,Borussia Dortmund,...,NaN,NaN,6.0,2.0,0,9.0,2.0,1,1.0,0
1,UFot0hit/,UFot0hit2,2,4,"Corner, Borussia Dortmund. Conceded by Dennis...",2,NaN,1,Borussia Dortmund,Hamburg SV,...,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0
2,UFot0hit/,UFot0hit3,3,4,"Corner, Borussia Dortmund. Conceded by Heiko ...",2,NaN,1,Borussia Dortmund,Hamburg SV,...,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0
3,UFot0hit/,UFot0hit4,4,7,Foul by Sven Bender (Borussia Dortmund).,3,NaN,1,Borussia Dortmund,Hamburg SV,...,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0
4,UFot0hit/,UFot0hit5,5,7,Gokhan Tore (Hamburg) wins a free kick in the ...,8,NaN,2,Hamburg SV,Borussia Dortmund,...,NaN,NaN,NaN,NaN,0,2.0,NaN,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941004,z5L2OT5E/,z5L2OT5E123,123,92,Lucas Torreira (Sampdoria) wins a free kick in...,8,NaN,2,Sampdoria,Atalanta,...,NaN,NaN,NaN,NaN,0,2.0,NaN,0,NaN,0
941005,z5L2OT5E/,z5L2OT5E124,124,93,"Corner, Sampdoria. Conceded by Andrea Masiello.",2,NaN,2,Sampdoria,Atalanta,...,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0
941006,z5L2OT5E/,z5L2OT5E125,125,93,Attempt missed. Fabio Quagliarella (Sampdoria)...,1,12.0,2,Sampdoria,Atalanta,...,NaN,NaN,8.0,2.0,0,9.0,1.0,1,3.0,0
941007,z5L2OT5E/,z5L2OT5E126,126,94,Alberto Grassi (Atalanta) wins a free kick on ...,8,NaN,1,Atalanta,Sampdoria,...,NaN,NaN,NaN,NaN,0,4.0,NaN,0,NaN,0


In [4]:
games_info

,id_odsp,link_odsp,adv_stats,date,league,season,country,ht,at,fthg,ftag,odd_h,odd_d,odd_a,odd_over,odd_under,odd_bts,odd_bts_n
0,UFot0hit/,/soccer/germany/bundesliga-2011-2012/dortmund-...,True,2011-08-05,D1,2012,germany,Borussia Dortmund,Hamburg SV,3,1,1.56,4.41,7.42,NaN,NaN,NaN,NaN
1,Aw5DflLH/,/soccer/germany/bundesliga-2011-2012/augsburg-...,True,2011-08-06,D1,2012,germany,FC Augsburg,SC Freiburg,2,2,2.36,3.60,3.40,NaN,NaN,NaN,NaN
2,bkjpaC6n/,/soccer/germany/bundesliga-2011-2012/werder-br...,True,2011-08-06,D1,2012,germany,Werder Bremen,Kaiserslautern,2,0,1.83,4.20,4.80,NaN,NaN,NaN,NaN
3,CzPV312a/,/soccer/france/ligue-1-2011-2012/paris-sg-lori...,True,2011-08-06,F1,2012,france,Paris Saint-Germain,Lorient,0,1,1.55,4.50,9.40,NaN,NaN,NaN,NaN
4,GUOdmtII/,/soccer/france/ligue-1-2011-2012/caen-valencie...,True,2011-08-06,F1,2012,france,Caen,Valenciennes,1,0,2.50,3.40,3.45,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10107,xAkY8l6R/,/soccer/italy/serie-a/genoa-crotone-xAkY8l6R/,True,2017-01-22,I1,2017,italy,Genoa,Crotone,2,2,1.97,4.35,8.00,1.95,2.03,2.03,1.86
10108,xSU9scI9/,/soccer/england/premier-league/chelsea-hull-ci...,True,2017-01-22,E0,2017,england,Chelsea,Hull,2,0,1.19,8.50,20.00,1.54,2.68,2.40,1.66
10109,xY7uZwOI/,/soccer/france/ligue-1/monaco-lorient-xY7uZwOI/,True,2017-01-22,F1,2017,france,AS Monaco,Lorient,4,0,1.32,6.24,11.50,1.53,3.08,1.80,2.25
10110,YyeGxMX8/,/soccer/spain/laliga/betis-gijon-YyeGxMX8/,True,2017-01-22,SP1,2017,spain,Real Betis,Sporting Gijon,0,0,1.74,4.07,5.90,2.20,1.89,2.05,1.86


In [5]:
# 2. Filter relevant columns
#    Incorporate shot_place, shot_outcome, side, etc.
relevant_columns = [
    'event_team',
    'event_type',
    'event_type2',
    'location',
    'assist_method',
    'situation',
    'bodypart',
    'shot_place',
    'shot_outcome',
    'side'
]
events = events[relevant_columns]
events

,event_team,event_type,event_type2,location,assist_method,situation,bodypart,shot_place,shot_outcome,side
0,Hamburg SV,1,12.0,9.0,1,1.0,2.0,6.0,2.0,2
1,Borussia Dortmund,2,NaN,NaN,0,NaN,NaN,NaN,NaN,1
2,Borussia Dortmund,2,NaN,NaN,0,NaN,NaN,NaN,NaN,1
3,Borussia Dortmund,3,NaN,NaN,0,NaN,NaN,NaN,NaN,1
4,Hamburg SV,8,NaN,2.0,0,NaN,NaN,NaN,NaN,2
...,...,...,...,...,...,...,...,...,...,...
941004,Sampdoria,8,NaN,2.0,0,NaN,NaN,NaN,NaN,2
941005,Sampdoria,2,NaN,NaN,0,NaN,NaN,NaN,NaN,2
941006,Sampdoria,1,12.0,9.0,1,3.0,1.0,8.0,2.0,2
941007,Atalanta,8,NaN,4.0,0,NaN,NaN,NaN,NaN,1


In [6]:
# Drop rows with missing critical values
critical_columns = ['event_team', 'event_type']
events = events.dropna(subset=critical_columns)
events

,event_team,event_type,event_type2,location,assist_method,situation,bodypart,shot_place,shot_outcome,side
0,Hamburg SV,1,12.0,9.0,1,1.0,2.0,6.0,2.0,2
1,Borussia Dortmund,2,NaN,NaN,0,NaN,NaN,NaN,NaN,1
2,Borussia Dortmund,2,NaN,NaN,0,NaN,NaN,NaN,NaN,1
3,Borussia Dortmund,3,NaN,NaN,0,NaN,NaN,NaN,NaN,1
4,Hamburg SV,8,NaN,2.0,0,NaN,NaN,NaN,NaN,2
...,...,...,...,...,...,...,...,...,...,...
941004,Sampdoria,8,NaN,2.0,0,NaN,NaN,NaN,NaN,2
941005,Sampdoria,2,NaN,NaN,0,NaN,NaN,NaN,NaN,2
941006,Sampdoria,1,12.0,9.0,1,3.0,1.0,8.0,2.0,2
941007,Atalanta,8,NaN,4.0,0,NaN,NaN,NaN,NaN,1


In [7]:
# One-hot encode categorical columns
categorical_columns = [
    'event_type',
    'event_type2',
    'location',
    'assist_method',
    'situation',
    'bodypart',
    'shot_place',
    'shot_outcome',
    'side'
]
encoder = OneHotEncoder(sparse_output=False)
encoded = encoder.fit_transform(events[categorical_columns])

In [8]:
# Create a DataFrame with the encoded features
encoded_feature_names = encoder.get_feature_names_out(categorical_columns)
encoded_features = pd.DataFrame(encoded, columns=encoded_feature_names)
encoded_features

,event_type_1,event_type_2,event_type_3,event_type_4,event_type_5,event_type_6,event_type_7,event_type_8,event_type_9,event_type_10,...,shot_place_12.0,shot_place_13.0,shot_place_nan,shot_outcome_1.0,shot_outcome_2.0,shot_outcome_3.0,shot_outcome_4.0,shot_outcome_nan,side_1,side_2
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
941005,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
941006,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
941007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [9]:
# Combine encoded features with the team column
data_encoded = pd.concat([events['event_team'].reset_index(drop=True), encoded_features], axis=1)
data_encoded

,event_team,event_type_1,event_type_2,event_type_3,event_type_4,event_type_5,event_type_6,event_type_7,event_type_8,event_type_9,...,shot_place_12.0,shot_place_13.0,shot_place_nan,shot_outcome_1.0,shot_outcome_2.0,shot_outcome_3.0,shot_outcome_4.0,shot_outcome_nan,side_1,side_2
0,Hamburg SV,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,Borussia Dortmund,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,Borussia Dortmund,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,Borussia Dortmund,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,Hamburg SV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941004,Sampdoria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
941005,Sampdoria,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
941006,Sampdoria,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
941007,Atalanta,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [11]:
# Group by team
team_aggregated = data_encoded.groupby('event_team').sum().reset_index()
team_aggregated

,event_team,event_type_1,event_type_2,event_type_3,event_type_4,event_type_5,event_type_6,event_type_7,event_type_8,event_type_9,...,shot_place_12.0,shot_place_13.0,shot_place_nan,shot_outcome_1.0,shot_outcome_2.0,shot_outcome_3.0,shot_outcome_4.0,shot_outcome_nan,side_1,side_2
0,AC Ajaccio,1106.0,449.0,1593.0,238.0,0.0,12.0,307.0,1534.0,365.0,...,28.0,30.0,4618.0,370.0,466.0,245.0,21.0,4613.0,3093.0,2622.0
1,AC Milan,3065.0,1225.0,2677.0,499.0,3.0,18.0,593.0,2840.0,543.0,...,68.0,72.0,8596.0,1030.0,1196.0,782.0,48.0,8577.0,6059.0,5574.0
2,AJ Auxerre,446.0,178.0,558.0,89.0,0.0,3.0,104.0,469.0,94.0,...,8.0,16.0,1540.0,156.0,188.0,96.0,6.0,1538.0,1067.0,917.0
3,AS Monaco,1665.0,727.0,1666.0,243.0,0.0,8.0,385.0,1736.0,285.0,...,41.0,41.0,5202.0,622.0,630.0,383.0,25.0,5190.0,3536.0,3314.0
4,AS Nancy Lorraine,1058.0,356.0,1260.0,160.0,0.0,7.0,232.0,1174.0,254.0,...,31.0,29.0,3527.0,347.0,495.0,197.0,18.0,3523.0,2330.0,2250.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,Villarreal,1849.0,798.0,2045.0,392.0,1.0,5.0,480.0,2085.0,465.0,...,45.0,50.0,6414.0,671.0,721.0,424.0,29.0,6403.0,4314.0,3934.0
138,Watford,678.0,250.0,725.0,129.0,1.0,4.0,164.0,639.0,82.0,...,15.0,23.0,2019.0,218.0,261.0,189.0,9.0,2017.0,1433.0,1261.0
139,Werder Bremen,2318.0,860.0,2645.0,375.0,0.0,10.0,490.0,2714.0,419.0,...,67.0,80.0,7675.0,804.0,921.0,550.0,38.0,7662.0,5386.0,4589.0
140,West Brom,1427.0,598.0,1328.0,230.0,0.0,6.0,356.0,1315.0,243.0,...,35.0,44.0,4110.0,420.0,598.0,392.0,12.0,4110.0,2846.0,2686.0


In [12]:
# Print columns names
team_aggregated.columns

Index(['event_team', 'event_type_1', 'event_type_2', 'event_type_3',
       'event_type_4', 'event_type_5', 'event_type_6', 'event_type_7',
       'event_type_8', 'event_type_9', 'event_type_10', 'event_type_11',
       'event_type2_12.0', 'event_type2_13.0', 'event_type2_14.0',
       'event_type2_15.0', 'event_type2_nan', 'location_1.0', 'location_2.0',
       'location_3.0', 'location_4.0', 'location_5.0', 'location_6.0',
       'location_7.0', 'location_8.0', 'location_9.0', 'location_10.0',
       'location_11.0', 'location_12.0', 'location_13.0', 'location_14.0',
       'location_15.0', 'location_16.0', 'location_17.0', 'location_18.0',
       'location_19.0', 'location_nan', 'assist_method_0', 'assist_method_1',
       'assist_method_2', 'assist_method_3', 'assist_method_4',
       'situation_1.0', 'situation_2.0', 'situation_3.0', 'situation_4.0',
       'situation_nan', 'bodypart_1.0', 'bodypart_2.0', 'bodypart_3.0',
       'bodypart_nan', 'shot_place_1.0', 'shot_place_2.0', '

In [16]:
# 8. Optionally drop "Not recorded" if it exists in new columns 
#    (e.g., if there's a shot_place_nan or shot_outcome_nan or location_19.0)
columns_to_drop = [
    col for col in team_aggregated.columns 
    if any(substr in col for substr in ['nan', 'Not recorded']) or col.endswith('19.0') 
]
team_aggregated.drop(columns=columns_to_drop, errors='ignore', inplace=True)

In [17]:
team_aggregated.columns

Index(['event_team', 'event_type_1', 'event_type_2', 'event_type_3',
       'event_type_4', 'event_type_5', 'event_type_6', 'event_type_7',
       'event_type_8', 'event_type_9', 'event_type_10', 'event_type_11',
       'event_type2_12.0', 'event_type2_13.0', 'event_type2_14.0',
       'event_type2_15.0', 'location_1.0', 'location_2.0', 'location_3.0',
       'location_4.0', 'location_5.0', 'location_6.0', 'location_7.0',
       'location_8.0', 'location_9.0', 'location_10.0', 'location_11.0',
       'location_12.0', 'location_13.0', 'location_14.0', 'location_15.0',
       'location_16.0', 'location_17.0', 'location_18.0', 'assist_method_0',
       'assist_method_1', 'assist_method_2', 'assist_method_3',
       'assist_method_4', 'situation_1.0', 'situation_2.0', 'situation_3.0',
       'situation_4.0', 'bodypart_1.0', 'bodypart_2.0', 'bodypart_3.0',
       'shot_place_1.0', 'shot_place_2.0', 'shot_place_3.0', 'shot_place_4.0',
       'shot_place_5.0', 'shot_place_6.0', 'shot_place_7.

In [21]:
team_aggregated

,event_team,event_type_1,event_type_2,event_type_3,event_type_4,event_type_5,event_type_6,event_type_7,event_type_8,event_type_9,...,shot_place_10.0,shot_place_11.0,shot_place_12.0,shot_place_13.0,shot_outcome_1.0,shot_outcome_2.0,shot_outcome_3.0,shot_outcome_4.0,side_1,side_2
0,AC Ajaccio,1106.0,449.0,1593.0,238.0,0.0,12.0,307.0,1534.0,365.0,...,90.0,25.0,28.0,30.0,370.0,466.0,245.0,21.0,3093.0,2622.0
1,AC Milan,3065.0,1225.0,2677.0,499.0,3.0,18.0,593.0,2840.0,543.0,...,224.0,63.0,68.0,72.0,1030.0,1196.0,782.0,48.0,6059.0,5574.0
2,AJ Auxerre,446.0,178.0,558.0,89.0,0.0,3.0,104.0,469.0,94.0,...,48.0,10.0,8.0,16.0,156.0,188.0,96.0,6.0,1067.0,917.0
3,AS Monaco,1665.0,727.0,1666.0,243.0,0.0,8.0,385.0,1736.0,285.0,...,78.0,38.0,41.0,41.0,622.0,630.0,383.0,25.0,3536.0,3314.0
4,AS Nancy Lorraine,1058.0,356.0,1260.0,160.0,0.0,7.0,232.0,1174.0,254.0,...,82.0,27.0,31.0,29.0,347.0,495.0,197.0,18.0,2330.0,2250.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,Villarreal,1849.0,798.0,2045.0,392.0,1.0,5.0,480.0,2085.0,465.0,...,118.0,45.0,45.0,50.0,671.0,721.0,424.0,29.0,4314.0,3934.0
138,Watford,678.0,250.0,725.0,129.0,1.0,4.0,164.0,639.0,82.0,...,31.0,19.0,15.0,23.0,218.0,261.0,189.0,9.0,1433.0,1261.0
139,Werder Bremen,2318.0,860.0,2645.0,375.0,0.0,10.0,490.0,2714.0,419.0,...,162.0,51.0,67.0,80.0,804.0,921.0,550.0,38.0,5386.0,4589.0
140,West Brom,1427.0,598.0,1328.0,230.0,0.0,6.0,356.0,1315.0,243.0,...,105.0,35.0,35.0,44.0,420.0,598.0,392.0,12.0,2846.0,2686.0


In [22]:
# Count games played for each team
# Combine home and away counts
games_played = games_info['ht'].value_counts() + games_info['at'].value_counts()
team_aggregated['games_played'] = team_aggregated['event_team'].map(games_played).fillna(1)  # Avoid division by zero

In [23]:
# Normalize event-related columns by games played
columns_to_normalize = [
    col for col in team_aggregated.columns 
    if col not in ['event_team', 'games_played']
]
for col in columns_to_normalize:
    team_aggregated[col] = team_aggregated[col] / team_aggregated['games_played']

In [24]:
team_aggregated

,event_team,event_type_1,event_type_2,event_type_3,event_type_4,event_type_5,event_type_6,event_type_7,event_type_8,event_type_9,...,shot_place_11.0,shot_place_12.0,shot_place_13.0,shot_outcome_1.0,shot_outcome_2.0,shot_outcome_3.0,shot_outcome_4.0,side_1,side_2,games_played
0,AC Ajaccio,9.701754,3.938596,13.973684,2.087719,0.000000,0.105263,2.692982,13.456140,3.201754,...,0.219298,0.245614,0.263158,3.245614,4.087719,2.149123,0.184211,27.131579,23.000000,114.0
1,AC Milan,14.595238,5.833333,12.747619,2.376190,0.014286,0.085714,2.823810,13.523810,2.585714,...,0.300000,0.323810,0.342857,4.904762,5.695238,3.723810,0.228571,28.852381,26.542857,210.0
2,AJ Auxerre,11.736842,4.684211,14.684211,2.342105,0.000000,0.078947,2.736842,12.342105,2.473684,...,0.263158,0.210526,0.421053,4.105263,4.947368,2.526316,0.157895,28.078947,24.131579,38.0
3,AS Monaco,12.333333,5.385185,12.340741,1.800000,0.000000,0.059259,2.851852,12.859259,2.111111,...,0.281481,0.303704,0.303704,4.607407,4.666667,2.837037,0.185185,26.192593,24.548148,135.0
4,AS Nancy Lorraine,11.020833,3.708333,13.125000,1.666667,0.000000,0.072917,2.416667,12.229167,2.645833,...,0.281250,0.322917,0.302083,3.614583,5.156250,2.052083,0.187500,24.270833,23.437500,96.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,Villarreal,10.812865,4.666667,11.959064,2.292398,0.005848,0.029240,2.807018,12.192982,2.719298,...,0.263158,0.263158,0.292398,3.923977,4.216374,2.479532,0.169591,25.228070,23.005848,171.0
138,Watford,11.300000,4.166667,12.083333,2.150000,0.016667,0.066667,2.733333,10.650000,1.366667,...,0.316667,0.250000,0.383333,3.633333,4.350000,3.150000,0.150000,23.883333,21.016667,60.0
139,Werder Bremen,12.395722,4.598930,14.144385,2.005348,0.000000,0.053476,2.620321,14.513369,2.240642,...,0.272727,0.358289,0.427807,4.299465,4.925134,2.941176,0.203209,28.802139,24.540107,187.0
140,West Brom,6.731132,2.820755,6.264151,1.084906,0.000000,0.028302,1.679245,6.202830,1.146226,...,0.165094,0.165094,0.207547,1.981132,2.820755,1.849057,0.056604,13.424528,12.669811,212.0


In [25]:
# 11. Create additional aggregate statistics / ratios
#     Example: total attempts, total fouls, total red cards, etc.

# Identify relevant sets of columns 
event_type_cols       = [c for c in columns_to_normalize if c.startswith('event_type_')]
shot_outcome_cols     = [c for c in columns_to_normalize if c.startswith('shot_outcome_')]
shot_place_cols       = [c for c in columns_to_normalize if c.startswith('shot_place_')]
card_cols             = [c for c in event_type_cols if any(x in c for x in ['yellow', 'red'])]
foul_cols             = [c for c in event_type_cols if 'Foul' in c]

In [26]:
# Total event_types sum
team_aggregated['total_events'] = team_aggregated[event_type_cols].sum(axis=1)

In [27]:
# Shots on target vs off target
team_aggregated['total_shots_on_target'] = team_aggregated.get('shot_outcome_1.0', 0.0)  # 1=On target
team_aggregated['total_shots_off_target'] = team_aggregated.get('shot_outcome_2.0', 0.0) # 2=Off target

In [28]:
# Example ratio: on-target shots vs total shots
total_shots = team_aggregated['total_shots_on_target'] + team_aggregated['total_shots_off_target']
team_aggregated['ratio_shots_on_target'] = np.where(
    total_shots > 0, 
    team_aggregated['total_shots_on_target'] / total_shots, 
    0
)

In [29]:
# Example ratio: total cards (yellow, red) per total fouls
team_aggregated['total_cards'] = team_aggregated[card_cols].sum(axis=1) if card_cols else 0
team_aggregated['total_fouls'] = team_aggregated[foul_cols].sum(axis=1) if foul_cols else 0
team_aggregated['ratio_cards_per_foul'] = np.where(
    team_aggregated['total_fouls'] > 0,
    team_aggregated['total_cards'] / team_aggregated['total_fouls'],
    0
)

In [30]:
# 12. Calculate percentage for each existing column relative to total_events
#     (Only do this for event-related columns, not the new ratio columns themselves!)
for col in columns_to_normalize:
    team_aggregated[f'percent_{col}'] = np.where(
        team_aggregated['total_events'] > 0,
        team_aggregated[col] / team_aggregated['total_events'],
        0
    )

C:\Users\stoic\AppData\Local\Temp\ipykernel_20172\2086384172.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team_aggregated[f'percent_{col}'] = np.where(
C:\Users\stoic\AppData\Local\Temp\ipykernel_20172\2086384172.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team_aggregated[f'percent_{col}'] = np.where(
C:\Users\stoic\AppData\Local\Temp\ipykernel_20172\2086384172.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Co

In [31]:
# 13. Add selected interaction terms
#     Extend or reduce these pairs as you wish. 
#     Consider including shot_outcome, shot_place, etc. if helpful.

event_type_columns    = [col for col in team_aggregated.columns if col.startswith('event_type_')]
location_columns      = [col for col in team_aggregated.columns if col.startswith('location_')]
assist_method_columns = [col for col in team_aggregated.columns if col.startswith('assist_method_')]
bodypart_columns      = [col for col in team_aggregated.columns if col.startswith('bodypart_')]
situation_columns     = [col for col in team_aggregated.columns if col.startswith('situation_')]
shot_outcome_columns  = [col for col in team_aggregated.columns if col.startswith('shot_outcome_')]
shot_place_columns    = [col for col in team_aggregated.columns if col.startswith('shot_place_')]


In [32]:
interaction_pairs = [
    (event_type_columns, location_columns),
    (event_type_columns, assist_method_columns),
    (location_columns, bodypart_columns),
    (location_columns, assist_method_columns),
    # Optionally add:
    (event_type_columns, shot_outcome_columns),
    (location_columns, shot_place_columns),
]

In [33]:
for category1, category2 in interaction_pairs:
    for col1, col2 in product(category1, category2):
        team_aggregated[f'{col1}_x_{col2}'] = team_aggregated[col1] * team_aggregated[col2]


C:\Users\stoic\AppData\Local\Temp\ipykernel_20172\615100895.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team_aggregated[f'{col1}_x_{col2}'] = team_aggregated[col1] * team_aggregated[col2]
C:\Users\stoic\AppData\Local\Temp\ipykernel_20172\615100895.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team_aggregated[f'{col1}_x_{col2}'] = team_aggregated[col1] * team_aggregated[col2]
C:\Users\stoic\AppData\Local\Temp\ipykernel_20172\615100895.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the res

In [34]:
# 14. Drop games_played (no longer needed for clustering features)
team_aggregated.drop(columns=['games_played'], inplace=True)

In [35]:
team_aggregated.columns

Index(['event_team', 'event_type_1', 'event_type_2', 'event_type_3',
       'event_type_4', 'event_type_5', 'event_type_6', 'event_type_7',
       'event_type_8', 'event_type_9',
       ...
       'location_18.0_x_shot_place_4.0', 'location_18.0_x_shot_place_5.0',
       'location_18.0_x_shot_place_6.0', 'location_18.0_x_shot_place_7.0',
       'location_18.0_x_shot_place_8.0', 'location_18.0_x_shot_place_9.0',
       'location_18.0_x_shot_place_10.0', 'location_18.0_x_shot_place_11.0',
       'location_18.0_x_shot_place_12.0', 'location_18.0_x_shot_place_13.0'],
      dtype='object', length=811)

In [36]:
team_aggregated

,event_team,event_type_1,event_type_2,event_type_3,event_type_4,event_type_5,event_type_6,event_type_7,event_type_8,event_type_9,...,location_18.0_x_shot_place_4.0,location_18.0_x_shot_place_5.0,location_18.0_x_shot_place_6.0,location_18.0_x_shot_place_7.0,location_18.0_x_shot_place_8.0,location_18.0_x_shot_place_9.0,location_18.0_x_shot_place_10.0,location_18.0_x_shot_place_11.0,location_18.0_x_shot_place_12.0,location_18.0_x_shot_place_13.0
0,AC Ajaccio,9.701754,3.938596,13.973684,2.087719,0.000000,0.105263,2.692982,13.456140,3.201754,...,0.018467,0.029086,0.020545,0.004848,0.031394,0.028163,0.020776,0.005771,0.006464,0.006925
1,AC Milan,14.595238,5.833333,12.747619,2.376190,0.014286,0.085714,2.823810,13.523810,2.585714,...,0.005782,0.006984,0.004966,0.001088,0.008095,0.006916,0.005079,0.001429,0.001542,0.001633
2,AJ Auxerre,11.736842,4.684211,14.684211,2.342105,0.000000,0.078947,2.736842,12.342105,2.473684,...,0.027008,0.038781,0.024931,0.004155,0.033241,0.032548,0.033241,0.006925,0.005540,0.011080
3,AS Monaco,12.333333,5.385185,12.340741,1.800000,0.000000,0.059259,2.851852,12.859259,2.111111,...,0.016900,0.022826,0.015802,0.002743,0.018875,0.020521,0.008560,0.004170,0.004499,0.004499
4,AS Nancy Lorraine,11.020833,3.708333,13.125000,1.666667,0.000000,0.072917,2.416667,12.229167,2.645833,...,0.016493,0.023872,0.020399,0.003906,0.028863,0.029297,0.017795,0.005859,0.006727,0.006293
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,Villarreal,10.812865,4.666667,11.959064,2.292398,0.005848,0.029240,2.807018,12.192982,2.719298,...,0.015492,0.025033,0.008208,0.002975,0.021032,0.022674,0.012106,0.004617,0.004617,0.005130
138,Watford,11.300000,4.166667,12.083333,2.150000,0.016667,0.066667,2.733333,10.650000,1.366667,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
139,Werder Bremen,12.395722,4.598930,14.144385,2.005348,0.000000,0.053476,2.620321,14.513369,2.240642,...,0.017501,0.021533,0.011324,0.003260,0.025823,0.019646,0.013898,0.004375,0.005748,0.006863
140,West Brom,6.731132,2.820755,6.264151,1.084906,0.000000,0.028302,1.679245,6.202830,1.146226,...,0.004183,0.005963,0.004850,0.000534,0.007520,0.007654,0.004672,0.001557,0.001557,0.001958


In [37]:
# 15. Scale features
scaler = StandardScaler()
features_to_scale = [col for col in team_aggregated.columns if col not in ['event_team']]
scaled_features = scaler.fit_transform(team_aggregated[features_to_scale])

In [38]:
# 16. Create final DataFrame for clustering
final_team_cluster_data = pd.DataFrame(scaled_features, columns=features_to_scale)
final_team_cluster_data['event_team'] = team_aggregated['event_team']

In [39]:
# 17. Save processed data
output_file = 'data/final_team_cluster_data2.csv'
final_team_cluster_data.to_csv(output_file, index=False)
print(f"Processed and normalized dataset saved to {output_file}")

Processed and normalized dataset saved to data/final_team_cluster_data2.csv
